In [ ]:
import tkinter as tk
from abc import ABC, abstractmethod
import copy
import random
from tkinter import messagebox
import sys
import re
import time
import math

# Evaluation scores for pieces
PIECE_SCORES = {
    'K': 1000, 'Q': 9, 'R': 5, 'B': 3, 'N': 3, 'P': 1,
    'k': -1000, 'q': -9, 'r': -5, 'b': -3, 'n': -3, 'p': -1
}

# Unicode chess pieces
UNICODE_PIECES = {
    ('white', 'King'): '♔',
    ('black', 'King'): '♚',
    ('white', 'Queen'): '♕',
    ('black', 'Queen'): '♛',
    ('white', 'Rook'): '♖',
    ('black', 'Rook'): '♜',
    ('white', 'Bishop'): '♗',
    ('black', 'Bishop'): '♝',
    ('white', 'Knight'): '♘',
    ('black', 'Knight'): '♞',
    ('white', 'Pawn'): '♙',
    ('black', 'Pawn'): '♟'
}

class Move:
    def __init__(self, src, dest, piece, captured=None, is_castling=False, rook_move=None, is_en_passant=False, en_passant_capture=None):
        self.src = src
        self.dest = dest
        self.piece = piece
        self.captured = captured
        self.is_castling = is_castling
        self.rook_move = rook_move
        self.is_en_passant = is_en_passant
        self.en_passant_capture = en_passant_capture

class CastlingRights:
    def __init__(self):
        self.white_kingside = True
        self.white_queenside = True
        self.black_kingside = True
        self.black_queenside = True

    def copy(self):
        new_rights = CastlingRights()
        new_rights.white_kingside = self.white_kingside
        new_rights.white_queenside = self.white_queenside
        new_rights.black_kingside = self.black_kingside
        new_rights.black_queenside = self.black_queenside
        return new_rights

class Piece(ABC):
    def __init__(self, color):
        self.color = color
        self.has_moved = False

    @abstractmethod
    def get_valid_moves(self, pos, board, castling_rights=None, last_move=None):
        pass

    def enemy(self, piece):
        return piece and piece.color != self.color

class King(Piece):
    def get_valid_moves(self, pos, board, castling_rights=None, last_move=None):
        x, y = pos
        moves = []
        for dx in [-1, 0, 1]:
            for dy in [-1, 0, 1]:
                if dx == 0 and dy == 0:
                    continue
                nx, ny = x + dx, y + dy
                if 0 <= nx < 8 and 0 <= ny < 8:
                    p = board[nx][ny]
                    if not p or self.enemy(p):
                        moves.append((nx, ny))

        if castling_rights and not self.has_moved:
            if (self.color == 'white' and castling_rights.white_kingside) or \
               (self.color == 'black' and castling_rights.black_kingside):
                if board[x][y+1] is None and board[x][y+2] is None:
                    rook = board[x][y+3]
                    if isinstance(rook, Rook) and rook.color == self.color and not rook.has_moved:
                        if not self.is_square_attacked((x, y), board, self.color) and \
                           not self.is_square_attacked((x, y+1), board, self.color) and \
                           not self.is_square_attacked((x, y+2), board, self.color):
                            moves.append((x, y+2))

            if (self.color == 'white' and castling_rights.white_queenside) or \
               (self.color == 'black' and castling_rights.black_queenside):
                if board[x][y-1] is None and board[x][y-2] is None and board[x][y-3] is None:
                    rook = board[x][y-4]
                    if isinstance(rook, Rook) and rook.color == self.color and not rook.has_moved:
                        if not self.is_square_attacked((x, y), board, self.color) and \
                           not self.is_square_attacked((x, y-1), board, self.color) and \
                           not self.is_square_attacked((x, y-2), board, self.color):
                            moves.append((x, y-2))

        return moves

    def is_square_attacked(self, pos, board, color):
        enemy_color = 'black' if color == 'white' else 'white'
        for x in range(8):
            for y in range(8):
                piece = board[x][y]
                if piece and piece.color == enemy_color:
                    if pos in piece.get_valid_moves((x, y), board):
                        return True
        return False

class Queen(Piece):
    def get_valid_moves(self, pos, board, castling_rights=None, last_move=None):
        return Rook(self.color).get_valid_moves(pos, board) + Bishop(self.color).get_valid_moves(pos, board)

class Rook(Piece):
    def get_valid_moves(self, pos, board, castling_rights=None, last_move=None):
        x, y = pos
        moves = []
        for dx, dy in [(-1,0),(1,0),(0,-1),(0,1)]:
            nx, ny = x, y
            while True:
                nx += dx
                ny += dy
                if 0 <= nx < 8 and 0 <= ny < 8:
                    p = board[nx][ny]
                    if not p:
                        moves.append((nx, ny))
                    elif self.enemy(p):
                        moves.append((nx, ny))
                        break
                    else:
                        break
                else:
                    break
        return moves

class Bishop(Piece):
    def get_valid_moves(self, pos, board, castling_rights=None, last_move=None):
        x, y = pos
        moves = []
        for dx, dy in [(-1,-1),(1,1),(-1,1),(1,-1)]:
            nx, ny = x, y
            while True:
                nx += dx
                ny += dy
                if 0 <= nx < 8 and 0 <= ny < 8:
                    p = board[nx][ny]
                    if not p:
                        moves.append((nx, ny))
                    elif self.enemy(p):
                        moves.append((nx, ny))
                        break
                    else:
                        break
                else:
                    break
        return moves

class Knight(Piece):
    def get_valid_moves(self, pos, board, castling_rights=None, last_move=None):
        x, y = pos
        moves = []
        for dx, dy in [(-2,-1),(-1,-2),(1,-2),(2,-1),(2,1),(1,2),(-1,2),(-2,1)]:
            nx, ny = x + dx, y + dy
            if 0 <= nx < 8 and 0 <= ny < 8:
                p = board[nx][ny]
                if not p or self.enemy(p):
                    moves.append((nx, ny))
        return moves

class Pawn(Piece):
    def get_valid_moves(self, pos, board, castling_rights=None, last_move=None):
        x, y = pos
        moves = []
        direction = -1 if self.color == 'white' else 1
        
        # Forward move
        if 0 <= x + direction < 8 and board[x + direction][y] is None:
            moves.append((x + direction, y))
            # Double move from starting position
            if ((self.color == 'white' and x == 6) or 
                (self.color == 'black' and x == 1)) and \
               board[x + 2*direction][y] is None:
                moves.append((x + 2*direction, y))
        
        # Diagonal captures
        for dy in [-1, 1]:
            nx, ny = x + direction, y + dy
            if 0 <= nx < 8 and 0 <= ny < 8:
                target = board[nx][ny]
                if target and self.enemy(target):
                    moves.append((nx, ny))
        
        # En passant
        if last_move and isinstance(last_move.piece, Pawn) and \
           abs(last_move.src[0] - last_move.dest[0]) == 2:
            last_x, last_y = last_move.dest
            if x == last_x and abs(y - last_y) == 1:
                moves.append((x + direction, last_y))
        
        return moves

class Player(ABC):
    def __init__(self, color):
        self.color = color

    @abstractmethod
    def get_move(self, board, castling_rights, last_move):
        pass

class HumanPlayer(Player):
    def get_move(self, board, castling_rights, last_move):
        return None

class AIPlayer(Player):
    def __init__(self, color, difficulty=2):
        super().__init__(color)
        self.difficulty = difficulty  # 1: Easy, 2: Medium, 3: Hard
        self.move_times = []
        self.nodes_evaluated = []
        
    def get_move(self, board, castling_rights, last_move):
        start_time = time.time()
        
        if self.difficulty == 1:
            # Easy: Random move
            moves = self.get_all_moves(board, self.color, castling_rights, last_move)
            if moves:
                move = random.choice(moves)
            else:
                return None
        elif self.difficulty == 2:
            # Medium: Genetic Algorithm for move selection
            move = self.genetic_algorithm_move(board, castling_rights, last_move)
        else:
            # Hard: Alpha-Beta Pruning with Minimax
            _, move = self.alpha_beta(board, castling_rights, last_move, 3, float('-inf'), float('inf'), self.color == 'white')
        
        end_time = time.time()
        self.move_times.append(end_time - start_time)
        return move
    
    def genetic_algorithm_move(self, board, castling_rights, last_move, population_size=20, generations=5):
        """Genetic Algorithm for move selection"""
        moves = self.get_all_moves(board, self.color, castling_rights, last_move)
        if not moves:
            return None
            
        # Initialize population
        population = [random.choice(moves) for _ in range(population_size)]
        
        for _ in range(generations):
            # Evaluate fitness
            fitness = []
            for move in population:
                new_board, new_rights = self.make_move(board, move, castling_rights)
                fitness.append(self.evaluate(new_board, new_rights))
            
            # Selection (tournament selection)
            new_population = []
            for _ in range(population_size):
                # Pick 4 random individuals and select the best
                candidates = random.sample(list(zip(population, fitness)), min(4, len(population)))
                best_move = max(candidates, key=lambda x: x[1] if self.color == 'white' else -x[1])[0]
                new_population.append(best_move)
            
            # Crossover (not applicable for single moves, so we just keep selection)
            population = new_population
            
            # Mutation (10% chance to replace with a random move)
            for i in range(len(population)):
                if random.random() < 0.1:
                    population[i] = random.choice(moves)
        
        # Select the best move from final population
        best_move = None
        best_score = float('-inf') if self.color == 'white' else float('inf')
        for move in population:
            new_board, new_rights = self.make_move(board, move, castling_rights)
            score = self.evaluate(new_board, new_rights)
            if (self.color == 'white' and score > best_score) or (self.color == 'black' and score < best_score):
                best_score = score
                best_move = move
        
        return best_move
    
    def alpha_beta(self, board, castling_rights, last_move, depth, alpha, beta, maximizing):
        """Alpha-Beta Pruning with Minimax"""
        if depth == 0:
            return self.evaluate(board, castling_rights), None
            
        moves = self.get_all_moves(board, 'white' if maximizing else 'black', castling_rights, last_move)
        if not moves:
            return self.evaluate(board, castling_rights), None
            
        best_move = None
        if maximizing:
            max_eval = float('-inf')
            for move in moves:
                self.nodes_evaluated.append(1)  # Track nodes evaluated
                new_board, new_rights = self.make_move(board, move, castling_rights)
                eval, _ = self.alpha_beta(new_board, new_rights, move, depth-1, alpha, beta, False)
                if eval > max_eval:
                    max_eval = eval
                    best_move = move
                alpha = max(alpha, eval)
                if beta <= alpha:
                    break
            return max_eval, best_move
        else:
            min_eval = float('inf')
            for move in moves:
                self.nodes_evaluated.append(1)  # Track nodes evaluated
                new_board, new_rights = self.make_move(board, move, castling_rights)
                eval, _ = self.alpha_beta(new_board, new_rights, move, depth-1, alpha, beta, True)
                if eval < min_eval:
                    min_eval = eval
                    best_move = move
                beta = min(beta, eval)
                if beta <= alpha:
                    break
            return min_eval, best_move

    def evaluate(self, board, castling_rights):
        """Enhanced evaluation function with positional considerations"""
        score = 0
        
        # Material score
        for row in board:
            for piece in row:
                if piece:
                    p = piece.__class__.__name__[0]
                    score += PIECE_SCORES.get(p.upper() if piece.color == 'white' else p.lower(), 0)

        # King safety
        for x in range(8):
            for y in range(8):
                piece = board[x][y]
                if isinstance(piece, King):
                    # King in center is bad in early/mid game
                    if 2 <= x <= 5 and 2 <= y <= 5:
                        score += -0.5 if piece.color == 'white' else 0.5
                    
                    # Castling rights bonus
                    if piece.color == 'white':
                        if castling_rights.white_kingside or castling_rights.white_queenside:
                            score += 1
                    else:
                        if castling_rights.black_kingside or castling_rights.black_queenside:
                            score -= 1
                    
                    # King shelter penalty (exposed king)
                    for dx in [-1, 0, 1]:
                        for dy in [-1, 0, 1]:
                            nx, ny = x + dx, y + dy
                            if 0 <= nx < 8 and 0 <= ny < 8:
                                if self.is_square_attacked((nx, ny), board, piece.color):
                                    score += -0.3 if piece.color == 'white' else 0.3

        # Center control
        center_squares = [(3, 3), (3, 4), (4, 3), (4, 4)]
        for x, y in center_squares:
            piece = board[x][y]
            if piece:
                score += 0.5 if piece.color == 'white' else -0.5

        # Mobility
        for x in range(8):
            for y in range(8):
                piece = board[x][y]
                if piece and not isinstance(piece, King):
                    moves = piece.get_valid_moves((x, y), board)
                    mobility_score = len(moves) * 0.1
                    score += mobility_score if piece.color == 'white' else -mobility_score

        # Pawn structure
        white_pawns = []
        black_pawns = []
        for x in range(8):
            for y in range(8):
                piece = board[x][y]
                if isinstance(piece, Pawn):
                    if piece.color == 'white':
                        white_pawns.append((x, y))
                    else:
                        black_pawns.append((x, y))
        
        # Doubled pawns penalty
        for y in range(8):
            white_count = sum(1 for x, py in white_pawns if py == y)
            black_count = sum(1 for x, py in black_pawns if py == y)
            if white_count > 1:
                score -= 0.5 * (white_count - 1)
            if black_count > 1:
                score += 0.5 * (black_count - 1)
        
        # Isolated pawns penalty
        for x, y in white_pawns:
            if not any((x, py) in white_pawns for py in [y-1, y+1] if 0 <= py < 8):
                score -= 0.5
        for x, y in black_pawns:
            if not any((x, py) in black_pawns for py in [y-1, y+1] if 0 <= py < 8):
                score += 0.5

        return score

    def is_square_attacked(self, pos, board, color):
        enemy_color = 'black' if color == 'white' else 'white'
        for x in range(8):
            for y in range(8):
                piece = board[x][y]
                if piece and piece.color == enemy_color:
                    if pos in piece.get_valid_moves((x, y), board):
                        return True
        return False

    def get_all_moves(self, board, color, castling_rights, last_move):
        moves = []
        for x in range(8):
            for y in range(8):
                piece = board[x][y]
                if piece and piece.color == color:
                    for dest in piece.get_valid_moves((x, y), board, castling_rights, last_move):
                        is_castling = False
                        rook_move = None
                        is_en_passant = False
                        en_passant_capture = None
                        if isinstance(piece, King) and abs(y - dest[1]) == 2:
                            is_castling = True
                            if dest[1] > y:
                                rook_move = ((x, 7), (x, 5))
                            else:
                                rook_move = ((x, 0), (x, 3))
                        elif isinstance(piece, Pawn) and dest[1] != y and board[dest[0]][dest[1]] is None:
                            is_en_passant = True
                            en_passant_capture = (dest[0] - (-1 if piece.color == 'white' else 1), dest[1])
                        moves.append(Move((x, y), dest, piece, is_castling=is_castling, rook_move=rook_move,
                                        is_en_passant=is_en_passant, en_passant_capture=en_passant_capture))
        return moves

    def make_move(self, board, move, castling_rights):
        new_board = copy.deepcopy(board)
        new_rights = castling_rights.copy()
        sx, sy = move.src
        dx, dy = move.dest
        piece = new_board[sx][sy]
        
        if move.is_castling:
            rook_src, rook_dest = move.rook_move
            new_board[rook_dest[0]][rook_dest[1]] = new_board[rook_src[0]][rook_src[1]]
            new_board[rook_src[0]][rook_src[1]] = None
            new_board[rook_dest[0]][rook_dest[1]].has_moved = True
        
        if move.is_en_passant:
            new_board[move.en_passant_capture[0]][move.en_passant_capture[1]] = None
        
        new_board[dx][dy] = piece
        new_board[sx][sy] = None
        piece.has_moved = True
        
        if isinstance(piece, King):
            if piece.color == 'white':
                new_rights.white_kingside = False
                new_rights.white_queenside = False
            else:
                new_rights.black_kingside = False
                new_rights.black_queenside = False
        elif isinstance(piece, Rook):
            if piece.color == 'white':
                if (sx, sy) == (7, 0):
                    new_rights.white_queenside = False
                elif (sx, sy) == (7, 7):
                    new_rights.white_kingside = False
            else:
                if (sx, sy) == (0, 0):
                    new_rights.black_queenside = False
                elif (sx, sy) == (0, 7):
                    new_rights.black_kingside = False

        return new_board, new_rights
    
    def get_performance_metrics(self):
        """Return performance metrics for the AI"""
        avg_time = sum(self.move_times)/len(self.move_times) if self.move_times else 0
        avg_nodes = sum(self.nodes_evaluated)/len(self.nodes_evaluated) if self.nodes_evaluated else 0
        return {
            'average_move_time': avg_time,
            'average_nodes_evaluated': avg_nodes,
            'total_moves': len(self.move_times)
        }

class ChessGame:
    def __init__(self, root):
        self.root = root
        self.root.title("Chess Game")
        
        # Make window responsive
        self.root.geometry("800x600")
        self.root.minsize(600, 400)  # Minimum window size
        self.root.state('zoomed')  # Start maximized
        
        # Initialize game state
        self.difficulty = 2  # Default medium difficulty
        self.show_main_menu()

    def show_main_menu(self):
        """Display the main menu screen"""
        self.clear_screen()
        
        # Main frame with responsive design
        self.main_frame = tk.Frame(self.root, bg='#2c3e50')
        self.main_frame.pack(fill=tk.BOTH, expand=True)
        
        # Container frame for centered content
        container = tk.Frame(self.main_frame, bg='#2c3e50')
        container.place(relx=0.5, rely=0.5, anchor='center')
        
        # Title with responsive font size
        title_font = ("Arial", min(36, int(self.root.winfo_screenheight()/20)), "bold")
        title_label = tk.Label(
            container,
            text="Chess Game",
            font=title_font,
            bg='#2c3e50',
            fg='white'
        )
        title_label.pack(pady=20)
        
        # Button style with responsive sizing
        button_font = ("Arial", min(24, int(self.root.winfo_screenheight()/30)))
        button_width = max(15, int(self.root.winfo_screenwidth()/80))
        
        # Play Button
        play_button = tk.Button(
            container,
            text="Play Game",
            command=self.show_difficulty_menu,
            font=button_font,
            bg='#3498db',
            fg='white',
            width=button_width,
            relief=tk.RAISED,
            borderwidth=3
        )
        play_button.pack(pady=10)
        
        # Quit Button
        quit_button = tk.Button(
            container,
            text="Quit",
            command=self.root.quit,
            font=button_font,
            bg='#e74c3c',
            fg='white',
            width=button_width,
            relief=tk.RAISED,
            borderwidth=3
        )
        quit_button.pack(pady=10)
        
        # Hover effects
        play_button.bind("<Enter>", lambda e: play_button.config(bg='#2980b9'))
        play_button.bind("<Leave>", lambda e: play_button.config(bg='#3498db'))
        quit_button.bind("<Enter>", lambda e: quit_button.config(bg='#c0392b'))
        quit_button.bind("<Leave>", lambda e: quit_button.config(bg='#e74c3c'))

    def show_difficulty_menu(self):
        """Display the difficulty selection screen"""
        self.clear_screen()
        
        # Main frame with responsive design
        self.main_frame = tk.Frame(self.root, bg='#2c3e50')
        self.main_frame.pack(fill=tk.BOTH, expand=True)
        
        # Container frame for centered content
        container = tk.Frame(self.main_frame, bg='#2c3e50')
        container.place(relx=0.5, rely=0.5, anchor='center')
        
        # Title with responsive font size
        title_font = ("Arial", min(36, int(self.root.winfo_screenheight()/20)), "bold")
        title_label = tk.Label(
            container,
            text="Select Difficulty",
            font=title_font,
            bg='#2c3e50',
            fg='white'
        )
        title_label.pack(pady=20)
        
        # Button style with responsive sizing
        button_font = ("Arial", min(20, int(self.root.winfo_screenheight()/30)))
        button_width = max(15, int(self.root.winfo_screenwidth()/80))
        
        # Difficulty buttons
        easy_button = tk.Button(
            container,
            text="Easy",
            command=lambda: self.set_difficulty_and_start(1),
            font=button_font,
            bg='#3498db',
            fg='white',
            width=button_width,
            relief=tk.RAISED,
            borderwidth=3
        )
        easy_button.pack(pady=5)
        
        medium_button = tk.Button(
            container,
            text="Medium",
            command=lambda: self.set_difficulty_and_start(2),
            font=button_font,
            bg='#3498db',
            fg='white',
            width=button_width,
            relief=tk.RAISED,
            borderwidth=3
        )
        medium_button.pack(pady=5)
        
        hard_button = tk.Button(
            container,
            text="Hard",
            command=lambda: self.set_difficulty_and_start(3),
            font=button_font,
            bg='#3498db',
            fg='white',
            width=button_width,
            relief=tk.RAISED,
            borderwidth=3
        )
        hard_button.pack(pady=5)
        
        # Back button with smaller size
        back_button = tk.Button(
            container,
            text="Back",
            command=self.show_main_menu,
            font=("Arial", min(16, int(self.root.winfo_screenheight()/40))),
            bg='#e74c3c',
            fg='white',
            width=max(10, int(self.root.winfo_screenwidth()/100)),
            relief=tk.RAISED,
            borderwidth=2
        )
        back_button.pack(pady=20)
        
        # Hover effects
        easy_button.bind("<Enter>", lambda e: easy_button.config(bg='#2980b9'))
        easy_button.bind("<Leave>", lambda e: easy_button.config(bg='#3498db'))
        medium_button.bind("<Enter>", lambda e: medium_button.config(bg='#2980b9'))
        medium_button.bind("<Leave>", lambda e: medium_button.config(bg='#3498db'))
        hard_button.bind("<Enter>", lambda e: hard_button.config(bg='#2980b9'))
        hard_button.bind("<Leave>", lambda e: hard_button.config(bg='#3498db'))
        back_button.bind("<Enter>", lambda e: back_button.config(bg='#c0392b'))
        back_button.bind("<Leave>", lambda e: back_button.config(bg='#e74c3c'))

    def set_difficulty_and_start(self, difficulty):
        """Set the difficulty level and start the game"""
        self.difficulty = difficulty
        self.start_game()

    def start_game(self):
        """Initialize and start the chess game"""
        self.clear_screen()
        self.init_game()
        self.setup_ui()
        self.log_event("Game started. White to move.")

    def init_game(self):
        """Initialize the game board and state"""
        self.board = [[None]*8 for _ in range(8)]
        
        # Set up pawns
        for i in range(8):
            self.board[1][i] = Pawn('black')
            self.board[6][i] = Pawn('white')
        
        # Set up other pieces
        pieces_order = [Rook, Knight, Bishop, Queen, King, Bishop, Knight, Rook]
        for i, piece_class in enumerate(pieces_order):
            self.board[0][i] = piece_class('black')
            self.board[7][i] = piece_class('white')
        
        # Game state variables
        self.turn = 'white'
        self.selected = None
        self.valid_moves = []
        self.game_over = False
        self.awaiting_promotion = False
        self.promotion_move = None
        self.castling_rights = CastlingRights()
        self.last_move = None
        self.ai = AIPlayer('black', difficulty=self.difficulty)

    def setup_ui(self):
        """Set up the chess game UI with responsive design"""
        # Main frame
        self.main_frame = tk.Frame(self.root)
        self.main_frame.pack(fill=tk.BOTH, expand=True)
        
        # Calculate sizes based on window dimensions
        screen_width = self.root.winfo_width()
        screen_height = self.root.winfo_height()
        
        # Chess board canvas - responsive size
        board_size = min(screen_height, int(screen_width * 0.7))
        self.canvas = tk.Canvas(self.main_frame, width=board_size, height=board_size)
        self.canvas.pack(side=tk.LEFT, fill=tk.BOTH, expand=True)
        
        # Status label
        self.status = tk.Label(self.main_frame, text="White to move", font=("Arial", 12))
        self.status.pack(side=tk.BOTTOM, fill=tk.X)
        
        # Control panel - responsive width
        control_panel_width = max(200, int(screen_width * 0.25))
        self.control_panel = tk.Frame(self.main_frame, bg='#2c3e50', width=control_panel_width)
        self.control_panel.pack(side=tk.RIGHT, fill=tk.Y)
        self.control_panel.pack_propagate(False)
        
        # Game status
        tk.Label(self.control_panel, text="Game Status", bg='#2c3e50', fg='white', 
                font=("Arial", 12, "bold")).pack(pady=10)
        self.status_box = tk.Text(self.control_panel, height=4, width=25, 
                                font=("Arial", 10), bg='#34495e', fg='white', wrap='word')
        self.status_box.pack(pady=5, padx=5, fill=tk.X)
        self.status_box.config(state='disabled')
        
        # AI Performance
        tk.Label(self.control_panel, text="AI Performance", bg='#2c3e50', fg='white', 
                font=("Arial", 12, "bold")).pack(pady=10)
        self.ai_perf_text = tk.Text(self.control_panel, height=4, width=25, 
                                   font=("Arial", 10), bg='#34495e', fg='white', wrap='word')
        self.ai_perf_text.pack(pady=5, padx=5, fill=tk.X)
        self.ai_perf_text.config(state='disabled')
        
        # Menu button
        menu_button = tk.Button(
            self.control_panel,
            text="Main Menu",
            command=self.show_main_menu,
            bg='#3498db',
            fg='white',
            font=("Arial", 14),
            relief=tk.RAISED,
            borderwidth=3
        )
        menu_button.pack(pady=20, fill=tk.X, padx=10)
        
        # New Game button
        new_game_button = tk.Button(
            self.control_panel,
            text="New Game",
            command=self.start_game,
            bg='#e74c3c',
            fg='white',
            font=("Arial", 14),
            relief=tk.RAISED,
            borderwidth=3
        )
        new_game_button.pack(pady=10, fill=tk.X, padx=10)
        
        # Hover effects
        menu_button.bind("<Enter>", lambda e: menu_button.config(bg='#2980b9'))
        menu_button.bind("<Leave>", lambda e: menu_button.config(bg='#3498db'))
        new_game_button.bind("<Enter>", lambda e: new_game_button.config(bg='#c0392b'))
        new_game_button.bind("<Leave>", lambda e: new_game_button.config(bg='#e74c3c'))
        
        # Bind click event
        self.canvas.bind("<Button-1>", self.click)
        
        # Make window resizable
        self.root.bind("<Configure>", self.on_window_resize)
        
        # Draw initial board
        self.draw_board()

    def on_window_resize(self, event):
        """Handle window resize events"""
        if hasattr(self, 'canvas') and self.canvas:
            # Redraw the board with new dimensions
            self.draw_board()

    def clear_screen(self):
        """Clear all widgets from the root window"""
        for widget in self.root.winfo_children():
            widget.destroy()

    def log_event(self, message):
        """Log game events to status box"""
        self.status_box.config(state='normal')
        self.status_box.delete(1.0, tk.END)
        self.status_box.insert(tk.END, message + "\n")
        self.status_box.config(state='disabled')

    def update_ai_performance(self):
        """Update the AI performance display"""
        metrics = self.ai.get_performance_metrics()
        text = f"AI Performance:\n"
        text += f"Avg move time: {metrics['average_move_time']:.2f}s\n"
        text += f"Avg nodes evaluated: {metrics['average_nodes_evaluated']:.0f}\n"
        text += f"Total moves: {metrics['total_moves']}"
        
        self.ai_perf_text.config(state='normal')
        self.ai_perf_text.delete(1.0, tk.END)
        self.ai_perf_text.insert(tk.END, text)
        self.ai_perf_text.config(state='disabled')

    def pos_to_coords(self, pos):
        """Convert algebraic notation to board coordinates"""
        if not re.match(r'^[a-h][1-8]$', pos):
            return None
        col = ord(pos[0].lower()) - ord('a')
        row = 8 - int(pos[1])
        if 0 <= row < 8 and 0 <= col < 8:
            return (row, col)
        return None

    def coords_to_pos(self, coords):
        """Convert board coordinates to algebraic notation"""
        row, col = coords
        return f"{chr(ord('a') + col)}{8 - row}"

    def is_check(self, color):
        """Check if king is in check"""
        king_pos = None
        for x in range(8):
            for y in range(8):
                piece = self.board[x][y]
                if isinstance(piece, King) and piece.color == color:
                    king_pos = (x, y)
                    break
            if king_pos:
                break

        enemy_color = 'black' if color == 'white' else 'white'
        for x in range(8):
            for y in range(8):
                piece = self.board[x][y]
                if piece and piece.color == enemy_color:
                    if king_pos in piece.get_valid_moves((x, y), self.board, last_move=self.last_move):
                        return True
        return False

    def is_checkmate(self, color):
        """Check if player is in checkmate"""
        if not self.is_check(color):
            return False
            
        for x in range(8):
            for y in range(8):
                piece = self.board[x][y]
                if piece and piece.color == color:
                    moves = piece.get_valid_moves((x, y), self.board, self.castling_rights, self.last_move)
                    for move in moves:
                        if self.is_valid_move((x, y), move):
                            return False
        return True

    def is_stalemate(self, color):
        """Check if game is in stalemate"""
        if self.is_check(color):
            return False
            
        for x in range(8):
            for y in range(8):
                piece = self.board[x][y]
                if piece and piece.color == color:
                    moves = piece.get_valid_moves((x, y), self.board, self.castling_rights, self.last_move)
                    for move in moves:
                        if self.is_valid_move((x, y), move):
                            return False
        return True

    def is_valid_move(self, src, dest):
        """Verify if move is valid (doesn't leave king in check)"""
        temp_board = copy.deepcopy(self.board)
        temp_rights = self.castling_rights.copy()
        sx, sy = src
        dx, dy = dest
        piece = temp_board[sx][sy]
        
        is_castling = isinstance(piece, King) and abs(sy - dy) == 2
        rook_move = None
        is_en_passant = False
        en_passant_capture = None
        if is_castling:
            if dy > sy:
                rook_move = ((sx, 7), (sx, 5))
            else:
                rook_move = ((sx, 0), (sx, 3))
        elif isinstance(piece, Pawn) and dy != sy and temp_board[dx][dy] is None:
            is_en_passant = True
            en_passant_capture = (dx - (-1 if piece.color == 'white' else 1), dy)
        
        temp_board[dx][dy] = piece
        temp_board[sx][sy] = None
        piece.has_moved = True
        
        if is_castling:
            rook_src, rook_dest = rook_move
            temp_board[rook_dest[0]][rook_dest[1]] = temp_board[rook_src[0]][rook_src[1]]
            temp_board[rook_src[0]][rook_src[1]] = None
            temp_board[rook_dest[0]][rook_dest[1]].has_moved = True
        
        if is_en_passant:
            temp_board[en_passant_capture[0]][en_passant_capture[1]] = None
        
        # Update castling rights
        if isinstance(piece, King):
            if piece.color == 'white':
                temp_rights.white_kingside = False
                temp_rights.white_queenside = False
            else:
                temp_rights.black_kingside = False
                temp_rights.black_queenside = False
        elif isinstance(piece, Rook):
            if piece.color == 'white':
                if (sx, sy) == (7, 0):
                    temp_rights.white_queenside = False
                elif (sx, sy) == (7, 7):
                    temp_rights.white_kingside = False
            else:
                if (sx, sy) == (0, 0):
                    temp_rights.black_queenside = False
                elif (sx, sy) == (0, 7):
                    temp_rights.black_kingside = False

        color = piece.color
        king_pos = None
        for x in range(8):
            for y in range(8):
                piece = temp_board[x][y]
                if isinstance(piece, King) and piece.color == color:
                    king_pos = (x, y)
                    break
            if king_pos:
                break

        enemy_color = 'black' if color == 'white' else 'white'
        for x in range(8):
            for y in range(8):
                piece = temp_board[x][y]
                if piece and piece.color == enemy_color:
                    if king_pos in piece.get_valid_moves((x, y), temp_board):
                        return False
        return True

    def execute_move(self, src, dest):
        # Execute a move from src to dest
        sx, sy = src
        dx, dy = dest
        piece = self.board[sx][sy]
        is_castling = isinstance(piece, King) and abs(sy - dy) == 2
        rook_move = None
        is_en_passant = False
        en_passant_capture = None
        captured_piece = self.board[dx][dy]
        src_pos = self.coords_to_pos((sx, sy))
        dest_pos = self.coords_to_pos((dx, dy))
        piece_name = piece.__class__.__name__
        if piece_name == "Knight":
            piece_name = "Knight"

        # Log move details
        if is_castling:
            if dy > sy:
                rook_move = ((sx, 7), (sx, 5))
                castling_type = "kingside"
            else:
                rook_move = ((sx, 0), (sx, 3))
                castling_type = "queenside"
            self.log_event(f"{piece.color.capitalize()} castles {castling_type}.")
        elif isinstance(piece, Pawn) and dy != sy and self.board[dx][dy] is None:
            is_en_passant = True
            en_passant_capture = (dx - (-1 if piece.color == 'white' else 1), dy)
            captured_piece = self.board[en_passant_capture[0]][en_passant_capture[1]]
            self.log_event(f"{piece_name} captures en passant at {dest_pos}.")
        elif captured_piece:
            captured_name = captured_piece.__class__.__name__
            if captured_name == "Knight":
                captured_name = "Knight"
            self.log_event(f"{piece_name} captures {captured_name} at {dest_pos}.")
        else:
            self.log_event(f"{piece_name} moves from {src_pos} to {dest_pos}.")

        move = Move(src, dest, piece, captured=captured_piece, is_castling=is_castling, rook_move=rook_move,
                    is_en_passant=is_en_passant, en_passant_capture=en_passant_capture)
        
        # Update board
        self.board[dx][dy] = piece
        self.board[sx][sy] = None
        piece.has_moved = True
        
        if is_castling:
            rook_src, rook_dest = rook_move
            self.board[rook_dest[0]][rook_dest[1]] = self.board[rook_src[0]][rook_src[1]]
            self.board[rook_src[0]][rook_src[1]] = None
            self.board[rook_dest[0]][rook_dest[1]].has_moved = True
        
        if is_en_passant:
            self.board[en_passant_capture[0]][en_passant_capture[1]] = None
        
        # Update castling rights
        if isinstance(piece, King):
            if piece.color == 'white':
                self.castling_rights.white_kingside = False
                self.castling_rights.white_queenside = False
            else:
                self.castling_rights.black_kingside = False
                self.castling_rights.black_queenside = False
        elif isinstance(piece, Rook):
            if piece.color == 'white':
                if src == (7, 0):
                    self.castling_rights.white_queenside = False
                elif src == (7, 7):
                    self.castling_rights.white_kingside = False
            else:
                if src == (0, 0):
                    self.castling_rights.black_queenside = False
                elif src == (0, 7):
                    self.castling_rights.black_kingside = False
        
        self.last_move = move
        self.selected = None
        self.valid_moves = []
        
        # Handle pawn promotion
        if isinstance(piece, Pawn) and dx == 0:
            self.awaiting_promotion = True
            self.show_promotion_dialog(dx, dy, 'white')
        else:
            self.turn = 'black'
            if self.is_checkmate('black'):
                self.game_over = True
                self.draw_board()
                self.show_game_over_dialog("White")
                return
            elif self.is_stalemate('black'):
                self.game_over = True
                self.draw_board()
                self.show_game_over_dialog("Draw")
                return
            elif self.is_check('black'):
                self.status.config(text="Black is in check! AI is thinking...")
                self.log_event("Black in check.")
            else:
                self.status.config(text="AI is thinking...")
            self.root.after(500, self.ai_move)

    def draw_board(self):
        """Draw the chess board with responsive sizing"""
        if not hasattr(self, 'canvas'):
            return
            
        self.canvas.delete("all")
        color = ["#f0d9b5", "#b58863"]  # Light and dark square colors
        
        # Calculate square size based on canvas dimensions
        canvas_width = self.canvas.winfo_width()
        canvas_height = self.canvas.winfo_height()
        square_size = min(canvas_width, canvas_height) // 8
        
        # Adjust offset to center the board
        x_offset = (canvas_width - square_size * 8) // 2
        y_offset = (canvas_height - square_size * 8) // 2
        
        # Draw board and pieces
        for x in range(8):
            # Row numbers
            self.canvas.create_text(
                x_offset // 2, 
                y_offset + x * square_size + square_size // 2, 
                text=str(8-x), 
                font=("Arial", max(10, square_size // 8))
            )
            
            for y in range(8):
                # Column letters (only for first row)
                if x == 0:
                    self.canvas.create_text(
                        x_offset + y * square_size + square_size // 2, 
                        y_offset // 2, 
                        text=chr(97+y), 
                        font=("Arial", max(10, square_size // 8))
                    )
                
                # Draw squares
                piece = self.board[x][y]
                x1 = x_offset + y * square_size
                y1 = y_offset + x * square_size
                x2 = x1 + square_size
                y2 = y1 + square_size
                
                if isinstance(piece, King) and self.is_check(piece.color):
                    self.canvas.create_rectangle(x1, y1, x2, y2, fill="red")
                else:
                    self.canvas.create_rectangle(x1, y1, x2, y2, fill=color[(x+y)%2])
                
                # Highlight selected piece
                if self.selected and self.selected == (x, y):
                    self.canvas.create_rectangle(x1, y1, x2, y2, outline="yellow", width=2)
                
                # Show valid moves
                if (x, y) in self.valid_moves:
                    center_x = x1 + square_size // 2
                    center_y = y1 + square_size // 2
                    radius = square_size // 6
                    self.canvas.create_oval(
                        center_x - radius, center_y - radius,
                        center_x + radius, center_y + radius,
                        fill="green", outline="green"
                    )
                
                # Draw pieces
                if piece:
                    unicode_piece = UNICODE_PIECES[(piece.color, piece.__class__.__name__)]
                    self.canvas.create_text(
                        x1 + square_size // 2, 
                        y1 + square_size // 2, 
                        text=unicode_piece, 
                        font=("Arial", int(square_size * 0.6))
                    )

    def click(self, event):
        # Handle mouse clicks on the board
        if self.game_over or self.awaiting_promotion:
            self.log_event("Cannot move: Game over or awaiting promotion.")
            return
            
        # Calculate square size and offset
        canvas_width = self.canvas.winfo_width()
        canvas_height = self.canvas.winfo_height()
        square_size = min(canvas_width, canvas_height) // 8
        x_offset = (canvas_width - square_size * 8) // 2
        y_offset = (canvas_height - square_size * 8) // 2
        
        # Get clicked coordinates
        x = (event.y - y_offset) // square_size
        y = (event.x - x_offset) // square_size
        
        # Validate click
        if not (0 <= x < 8 and 0 <= y < 8) or self.turn != 'white':
            self.log_event("Cannot select: Invalid click or not white's turn.")
            return
            
        if self.selected:
            if (x, y) in self.valid_moves:
                self.execute_move(self.selected, (x, y))
            else:
                self.log_event(f"Invalid move to {self.coords_to_pos((x, y))}.")
            self.selected = None
            self.valid_moves = []
            
        else:
            piece = self.board[x][y]
            if piece and piece.color == 'white':
                self.selected = (x, y)
                all_moves = piece.get_valid_moves((x, y), self.board, self.castling_rights, self.last_move)
                self.valid_moves = [(dx, dy) for dx, dy in all_moves if self.is_valid_move((x, y), (dx, dy))]
                moves_str = ", ".join(self.coords_to_pos(move) for move in self.valid_moves)
                piece_name = piece.__class__.__name__
                if piece_name == "Knight":
                    piece_name = "Knight"
                self.log_event(f"{piece_name} selected at {self.coords_to_pos((x, y))}.")
            else:
                self.log_event(f"No white piece at {self.coords_to_pos((x, y))}.")
                
        self.draw_board()

    def show_promotion_dialog(self, x, y, color):
        """Display pawn promotion dialog"""
        self.promotion_window = tk.Toplevel(self.root)
        self.promotion_window.title("Pawn Promotion")
        self.promotion_window.geometry("300x400")
        self.promotion_window.configure(bg='#2c3e50')
        
        title_label = tk.Label(self.promotion_window, 
                             text="Choose Your Promotion Piece",
                             font=("Arial", 16, "bold"),
                             bg='#2c3e50',
                             fg='white')
        title_label.pack(pady=20)
        
        button_style = {
            'font': ('Arial', 14),
            'width': 15,
            'height': 2,
            'bg': '#3498db',
            'fg': 'white',
            'activebackground': '#2980b9',
            'activeforeground': 'white',
            'relief': 'raised',
            'bd': 3
        }
        
        def make_choice(piece_class):
            piece_name = piece_class.__name__
            if piece_name == "Knight":
                piece_name = "Knight"
            self.board[x][y] = piece_class(color)
            self.promotion_window.destroy()
            self.promotion_window = None
            self.awaiting_promotion = False
            self.log_event(f"{color.capitalize()} pawn promoted to {piece_name} at {self.coords_to_pos((x, y))}.")
            
            if color == 'black':
                if self.is_checkmate('white'):
                    self.game_over = True
                    self.draw_board()
                    self.show_game_over_dialog("Black")
                elif self.is_stalemate('white'):
                    self.game_over = True
                    self.draw_board()
                    self.show_game_over_dialog("Draw")
                else:
                    self.turn = 'white'
                    self.status.config(text="White to move")
                    self.draw_board()
                    if self.is_check('white'):
                        self.status.config(text="White is in check! Your move")
                        self.log_event("White in check.")
            else:
                self.turn = 'black'
                self.root.after(500, self.ai_move)
        
        # Create buttons for promotion options
        pieces = [
            ("Queen", Queen, UNICODE_PIECES[(color, 'Queen')]),
            ("Rook", Rook, UNICODE_PIECES[(color, 'Rook')]),
            ("Bishop", Bishop, UNICODE_PIECES[(color, 'Bishop')]),
            ("Knight", Knight, UNICODE_PIECES[(color, 'Knight')])
        ]
        
        for piece_name, piece_class, unicode_char in pieces:
            button_frame = tk.Frame(self.promotion_window, bg='#2c3e50')
            button_frame.pack(pady=10)
            
            button = tk.Button(button_frame,
                             text=f"{unicode_char} {piece_name}",
                             command=lambda pc=piece_class: make_choice(pc),
                             **button_style)
            button.pack()
            
            def on_enter(e, btn=button):
                btn.configure(bg='#2980b9')
            
            def on_leave(e, btn=button):
                btn.configure(bg='#3498db')
                
            button.bind("<Enter>", on_enter)
            button.bind("<Leave>", on_leave)
        
        # Center the promotion window
        self.promotion_window.update_idletasks()
        width = self.promotion_window.winfo_width()
        height = self.promotion_window.winfo_height()
        x_offset = (self.promotion_window.winfo_screenwidth() // 2) - (width // 2)
        y_offset = (self.promotion_window.winfo_screenheight() // 2) - (height // 2)
        self.promotion_window.geometry(f"+{x_offset}+{y_offset}")
        self.log_event(f"{color.capitalize()} pawn promotion at {self.coords_to_pos((x, y))}.")

    def show_game_over_dialog(self, result):
        """Show game over dialog"""
        if result == "Draw":
            message = "Stalemate! The game is a draw."
        else:
            message = f"Checkmate! {result} wins!"
        
        self.log_event(message)
        self.game_over = True
        messagebox.showinfo("Game Over", message + "\nClick 'New Game' to play again")

    def ai_move(self):
        # Handle AI's move
        move = self.ai.get_move(self.board, self.castling_rights, self.last_move)
        if move:
            sx, sy = move.src
            dx, dy = move.dest
            piece = self.board[sx][sy]
            src_pos = self.coords_to_pos((sx, sy))
            dest_pos = self.coords_to_pos((dx, dy))
            piece_name = piece.__class__.__name__
            if piece_name == "Knight":
                piece_name = "Knight"
            captured_piece = self.board[dx][dy]
            
            # Log AI move details
            if move.is_castling:
                rook_src, rook_dest = move.rook_move
                self.board[rook_dest[0]][rook_dest[1]] = self.board[rook_src[0]][rook_src[1]]
                self.board[rook_src[0]][rook_src[1]] = None
                self.board[rook_dest[0]][rook_dest[1]].has_moved = True
                castling_type = "kingside" if dy > sy else "queenside"
                self.log_event(f"Black castles {castling_type}.")
            elif move.is_en_passant:
                self.board[move.en_passant_capture[0]][move.en_passant_capture[1]] = None
                self.log_event(f"{piece_name} captures en passant at {dest_pos}.")
            elif captured_piece:
                captured_name = captured_piece.__class__.__name__
                if captured_name == "Knight":
                    captured_name = "Knight"
                self.log_event(f"{piece_name} captures {captured_name} at {dest_pos}.")
            else:
                self.log_event(f"{piece_name} moves from {src_pos} to {dest_pos}.")
            
            # Update board
            self.board[dx][dy] = piece
            self.board[sx][sy] = None
            piece.has_moved = True
            
            # Update castling rights
            if isinstance(piece, King):
                if piece.color == 'white':
                    self.castling_rights.white_kingside = False
                    self.castling_rights.white_queenside = False
                else:
                    self.castling_rights.black_kingside = False
                    self.castling_rights.black_queenside = False
            elif isinstance(piece, Rook):
                if piece.color == 'white':
                    if (sx, sy) == (7, 0):
                        self.castling_rights.white_queenside = False
                    elif (sx, sy) == (7, 7):
                        self.castling_rights.white_kingside = False
                else:
                    if (sx, sy) == (0, 0):
                        self.castling_rights.black_queenside = False
                    elif (sx, sy) == (0, 7):
                        self.castling_rights.black_kingside = False
            
            self.last_move = move
            
            # Handle pawn promotion for AI
            if isinstance(piece, Pawn) and dx == 7:
                self.board[dx][dy] = Queen('black')  # AI promotes to Queen
                self.log_event(f"Black pawn promoted to Queen at {dest_pos}.")
                self.awaiting_promotion = False
                
                # Check game state after promotion
                if self.is_checkmate('white'):
                    self.game_over = True
                    self.draw_board()
                    self.show_game_over_dialog("Black")
                    return
                elif self.is_stalemate('white'):
                    self.game_over = True
                    self.draw_board()
                    self.show_game_over_dialog("Draw")
                    return
                elif self.is_check('white'):
                    self.status.config(text="White is in check! Your move")
                    self.log_event("White in check.")
                else:
                    self.status.config(text="White to move")
                
                self.turn = 'white'
                self.draw_board()
                return
                
            # Check game state
            if self.is_checkmate('white'):
                self.game_over = True
                self.draw_board()
                self.show_game_over_dialog("Black")
                return
            elif self.is_stalemate('white'):
                self.game_over = True
                self.draw_board()
                self.show_game_over_dialog("Draw")
                return
            elif self.is_check('white'):
                self.status.config(text="White is in check! Your move")
                self.log_event("White in check.")
            else:
                self.status.config(text="White to move")
            
            # Update AI performance metrics
            self.update_ai_performance()
                
        self.turn = 'white'
        self.draw_board()

if __name__ == '__main__':
    # Start the Tkinter application
    root = tk.Tk()
    game = ChessGame(root)
    root.mainloop()